In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
    .appName("twitter_silver")\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/03/01 16:22:47 WARN Utils: Your hostname, cmatheus resolves to a loopback address: 127.0.1.1; using 172.30.136.204 instead (on interface eth0)
23/03/01 16:22:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/01 16:22:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_tweet = spark.read.json("/mnt/e/alura/alura_airflow_Twitter/data_lake/silver/twitter_extract_dataScience/tweets")

In [4]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- impression_count: long (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [5]:
from pyspark.sql import functions as f

In [6]:
tweet_conversas = df_tweet.alias("tweet")\
                    .groupBy(f.to_date("created_at").alias("created_at"))\
                    .agg(
                            f.countDistinct("author_id").alias("n_tweets"),
                            f.sum("like_count").alias("n_likes"),
                            f.sum("quote_count").alias("n_quote_count"),
                            f.sum("reply_count").alias("n_reply_count"),
                            f.sum("retweet_count").alias("n_retweet_count"),
                    ).withColumn("weekday", f.date_format("created_at","E"))

In [7]:
tweet_conversas.show()

+----------+--------+-------+-------------+-------------+---------------+-------+
|created_at|n_tweets|n_likes|n_quote_count|n_reply_count|n_retweet_count|weekday|
+----------+--------+-------+-------------+-------------+---------------+-------+
|2023-02-27|     491|   1396|           28|          114|          59354|    Mon|
|2023-02-28|     452|   1800|           36|           78|          28390|    Tue|
+----------+--------+-------+-------------+-------------+---------------+-------+

